## Install and Import dependencies

In [1]:
!pip install tensorflow-gpu==1.15.0 tensorflow==1.15.0 gym-anytrading gym stable-baselines numpy

In [2]:
pip install py

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Gym
import gym
import gym_anytrading

# Stable baselines
from stable_baselines import A2C
from stable_baselines.common.vec_env import DummyVecEnv
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#custom Indicator
from gym_anytrading.envs import StocksEnv
from finta import TA

ModuleNotFoundError: No module named 'ale_py._ale_py'

## Data Processing

In [ ]:
df = pd.read_csv("data/Binance_BTCUSDT_1h.csv", skiprows=1)

In [ ]:
df = df.drop(columns=["Volume USDT", "tradecount", "symbol"], axis=1)

In [ ]:
df.head()

In [ ]:
nunix = []

for i in df["unix"]:
    if i < 1000000000000:
        r = i * 1000
    else:
        r = i
    nunix.append(r)
    
df["unix"] = nunix

In [ ]:
df["date"] = pd.to_datetime(df["unix"],unit="ms")

In [ ]:
df = df.drop(columns=["unix"], axis=1)

In [ ]:
df = df.rename(columns={"Volume BTC":"volume"})

In [ ]:
df.head()

In [ ]:
df.set_index("date", inplace=True)
df.sort_index()

In [ ]:
df = df.rename(columns={"date":"Date","open":"Open", "high":"High", "low":"Low", "close":"Close"})

In [ ]:
df.head()

In [ ]:
df.to_csv("data/BTC_USDT.csv")

In [ ]:
df = df.sort_index()

In [ ]:
df.head()

## 2. Add Custom Indicators

In [ ]:
!pip install finta

In [ ]:
from gym_anytrading.envs import StocksEnv
from finta import TA

In [ ]:
df["RSI"] = TA.RSI(df)
df["VWAP"] = TA.VWAP(df)

### 3.  Build Environment

In [ ]:
env = gym.make("stocks-v0", df=df, frame_bound=(5,100), window_size=5)

In [ ]:
env.signal_features

In [ ]:
state = env.reset()
while True:
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    if done:
        print("info", info)
        break
        
plt.figure(figsize=(150,50))
plt.cla()
env.render_all()
plt.show()

## Train Environment

In [ ]:
env_maker = lambda:gym.make("stocks-v0", df = df, frame_bound=(5,100), window_size=5)
env= DummyVecEnv([env_maker])

In [ ]:
modal = A2C("MlpLstmnPolicy", env, verbose=1)
modal.learn(total_timesteps=100000)

## Evaluation

In [ ]:
env = gym.make("stocks-v0", df=df, frame_bound=(5,100), window_size=5)
obs = env.reset()
while True:
    obs = obs[np.newaxis, ...]
    action, _states = modal.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break
        

In [ ]:
plt.figure(figsize=(150,50))
plt.cla()
env.render_all()
plt.show()